In [3]:
import json
import Database_connector
import pandas as pd
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import torch
import numpy as np
from Database_Settings import DB_SETTINGS
import Database_connector
from mysql.connector import (connection)
from PIL import Image

PICKLED_DATA_FOLDER = "Pickled_Data/"
PICKLED_PICTURES_FILE = "pictures_embeddings_resnet152_df.pkl"
PICKLED_OUTFIRS_FILE = "active_outfits_df.pkl"

PICTURES_DROP_COLUMNS = ["contentType", "status", "displayOrder", "sourceURL", "embeddings"]
#PICTURES_DROP_COLUMNS = []
OUTFITS_DROP_COLUMNS = ['owner', 'name', 'brand', 'isPublic', 'isDeleted', 'meta.validFrom', 'meta.validTo', "Outfit_size"]


def get_active_outfits():
    cnx = connection.MySQLConnection(**DB_SETTINGS)
    db_connection = Database_connector.Db_Connection()
    cursor = cnx.cursor(dictionary=True)
    #tag_query = "SELECT * FROM Outfits WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
    tag_query = "SELECT `Outfits`.`id` AS `id`, `Outfits`.`owner` AS `owner`, `Outfits`.`name` AS `name`, `Outfits`.`brand` AS `brand`, `Outfits`.`isPublic` AS `isPublic`, `Outfits`.`isDeleted` AS `isDeleted`, `Outfits`.`timeCreated` AS `timeCreated`, `Outfits`.`timeUpdated` AS `timeUpdated`, `Outfits`.`pricePerWeek` AS `pricePerWeek`, `Outfits`.`pricePerMonth` AS `pricePerMonth`, `Outfits`.`type` AS `type`, `Outfits`.`keywords` AS `keywords`, `Outfits`.`retailPrice` AS `retailPrice`, `Outfits`.`meta.validFrom` AS `meta.validFrom`, `Outfits`.`meta.validTo` AS `meta.validTo` FROM `Outfits` WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
    cursor.execute(tag_query)
    outfit_results = cursor.fetchall()

    outfits_df_db = pd.DataFrame([list(order_dict.values()) for order_dict in outfit_results], columns=list(outfit_results[0].keys()))
    return outfits_df_db

def get_pictures_df(path):
    pictures_df = pd.read_pickle(path)
    #pictures_df = pictures_df.drop(columns=PICTURES_DROP_COLUMNS)
    pictures_df = pictures_df.drop(columns=pictures_df.columns[2:])
    return pictures_df

def get_outfits_df(path):
    outfits_df = pd.read_pickle(path)
    outfits_df = outfits_df.drop(columns=OUTFITS_DROP_COLUMNS)
    return outfits_df

def prepare_data(pictures_df_path = PICKLED_DATA_FOLDER + PICKLED_PICTURES_FILE, outfits_df_path = PICKLED_DATA_FOLDER + PICKLED_OUTFIRS_FILE):
    pictures_df = get_pictures_df(pictures_df_path)
    outfits_df = get_outfits_df(outfits_df_path)
    return pictures_df, outfits_df

def picture_exists(picture_id, pictures_dir_path):
    return os.path.isfile(pictures_dir_path + os.sep + picture_id + ".jpg")

def find_missing_pictures(pictures_df, pictures_dir_path):
    pictures_df["file_exists"] = pictures_df["id"].apply(lambda x : picture_exists(x, pictures_dir_path))
    return pictures_df["file_exists"].value_counts()

In [5]:
import torch
import torchvision

def load_resnet152_model():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', weights=torchvision.models.ResNet152_Weights.IMAGENET1K_V1)
    embedding_model = torch.nn.Sequential(*list(model.children())[:-1])
    embedding_model.eval()
    return embedding_model

def load_resnet101_model():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1)
    embedding_model = torch.nn.Sequential(*list(model.children())[:-1])
    embedding_model.eval()
    return embedding_model

def load_resnet50_model():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
    embedding_model = torch.nn.Sequential(*list(model.children())[:-1])
    embedding_model.eval()
    return embedding_model

def load_resnet34_model():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
    embedding_model = torch.nn.Sequential(*list(model.children())[:-1])
    embedding_model.eval()
    return embedding_model

def load_resnet18_model():
    model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
    embedding_model = torch.nn.Sequential(*list(model.children())[:-1])
    embedding_model.eval()
    return embedding_model


In [8]:
tag_weights = pd.read_pickle("Pickled_Data/weights_df.pkl")
tag_weights

,category,weight,sum
0,Sleeve,0.496527,46.290323
1,Waist,0.428134,42.142857
2,Material,0.268592,47.159574
3,Length,0.542224,45.185185
4,Pattern,0.500707,43.588235
5,Brand,0.198210,51.013514
6,Fit,0.454452,47.192308
7,Neckline,0.431552,43.078947
8,Category,0.435782,50.550725
9,Size,0.479231,47.977778


In [18]:
import torchvision
from torchvision.models import get_model_weights

# print model weight names
get_model_weights("Resnet152")
torchvision.models.ResNet18_Weights.IMAGENET1K_V1

<enum 'ResNet50_Weights'>

In [24]:
import pandas as pd

original_pickle_path = "Pickled_Data/pictures_embeddings_resnet152_df.pkl"
pd.read_pickle(original_pickle_path)

,id,owner,contentType,status,displayOrder,sourceURL,embeddings
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,image/jpeg,ACTIVE,2,https://cdn.shopify.com/s/files/1/0424/9927/69...,"[tensor(0.2495), tensor(1.2492), tensor(0.), t..."
1,picture.000cf715019a4a02aaf0dc479212195a,outfit.f11bdc25d87946f8b831b608bc8fb574,image/jpeg,ACTIVE,1,https://storage.googleapis.com/batch-uploads-p...,"[tensor(0.2230), tensor(1.3119), tensor(0.4345..."
2,picture.0010c2e161154d6893734981d5455e76,outfit.9387d05b47f906c5,image/jpeg,ACTIVE,0,None,"[tensor(0.1586), tensor(0.8255), tensor(0.9512..."
3,picture.00135ce8b1a04c5daa60cf7bdd99bcd5,outfit.12ae28ab0dc5494c98a1de2f8ce04b79,image/jpeg,ACTIVE,1,https://storage.googleapis.com/batch-uploads-p...,"[tensor(0.2306), tensor(0.3835), tensor(0.1205..."
4,picture.001a58ea68da426384567b8cccc0c8a6,outfit.e989b8cb4a814d97b642e1cb326f47e6,image/png,ACTIVE,0,None,"[tensor(0.0074), tensor(2.0541), tensor(0.5028..."
...,...,...,...,...,...,...,...
18055,picture.ffe927a5dc06491fb73958a896ac2b19,outfit.0263a299144e4c89b68dd66990e43d65,image/jpeg,ACTIVE,1,https://storage.googleapis.com/batch-uploads-p...,"[tensor(0.1538), tensor(1.2303), tensor(0.1667..."
18056,picture.fff4edd997914149927603fb0a9a3009,outfit.4779ad6771a746408fb5d492ae345d18,image/jpeg,ACTIVE,2,https://storage.googleapis.com/batch-uploads-p...,"[tensor(0.4513), tensor(1.9715), tensor(0.1843..."
18057,picture.fff842f598dd4c3d9c80d2c1e00c4a26,outfit.c9ba76ae33c44aa5a06b23a4e4229fb1,image/jpeg,ACTIVE,0,None,"[tensor(0.0028), tensor(0.5193), tensor(0.1181..."
18058,picture.fffa8417a83642e0aba3c8187e80a3b9,outfit.225b573681f9446a8b5dc126add381b8,image/jpeg,ACTIVE,2,https://storage.googleapis.com/batch-uploads-p...,"[tensor(0.4813), tensor(1.8617), tensor(0.0046..."


In [25]:
pictures_df, outfits_df = prepare_data(pictures_df_path="../FREja_dataset_processing/pictures_embeddings_resnet152_df.pkl"
                                       , outfits_df_path="../FREja_dataset_processing/active_outfits_df.pkl")
pictures_df.head()

,id,owner
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc
1,picture.000cf715019a4a02aaf0dc479212195a,outfit.f11bdc25d87946f8b831b608bc8fb574
2,picture.0010c2e161154d6893734981d5455e76,outfit.9387d05b47f906c5
3,picture.00135ce8b1a04c5daa60cf7bdd99bcd5,outfit.12ae28ab0dc5494c98a1de2f8ce04b79
4,picture.001a58ea68da426384567b8cccc0c8a6,outfit.e989b8cb4a814d97b642e1cb326f47e6


In [26]:
EMBEDDINGS_SAVE_DIR = "Pickled_Data/Embeddings/"
PICTURES_DIR_PATH = "../FREja_dataset_processing/Filtered_pictures/"
DF_SPLITS = 200
RUN_CONFIGS = [
    {
        "model" : load_resnet152_model,
        "save_file_name" : "resnet152_v1Embeddings_df.pkl"
    },
    {
        "model" : load_resnet101_model,
        "save_file_name" : "resnet101_v1Embeddings_df.pkl"
    },
    {
        "model" : load_resnet50_model,
        "save_file_name" : "resnet50_v1Embeddings_df.pkl"
    },
    {
        "model" : load_resnet34_model,
        "save_file_name" : "resnet34_v1Embeddings_df.pkl"
    },
    {
        "model" : load_resnet18_model,
        "save_file_name" : "resnet18_v1Embeddings_df.pkl"
    }
]

if not os.path.isdir(EMBEDDINGS_SAVE_DIR):
    os.mkdir(EMBEDDINGS_SAVE_DIR)

In [27]:
find_missing_pictures(pictures_df, PICTURES_DIR_PATH)

True    18060
Name: file_exists, dtype: int64

In [28]:
from torchvision import transforms
import copy
from IPython.display import display
import ipywidgets
from tqdm import tqdm

#file_path = "C:\\Datasets\\FJONG_image_dataset\\FWSS PHOTOS 20-6-22\\Ain't No Mountain - 2 copy.jpg"
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_image(image_id):
    file_name = image_id + ".jpg"
    file_path = PICTURES_DIR_PATH + file_name
    image = Image.open(file_path)
    if image.mode != "RGB":
        print(f"Found image with mode: {image.mode}, id: {image_id}")
        image = image.convert("RGB")
    return image

def image_to_embedding(preprocess_transform, model_e, image_id, d_widget, index_num):
    d_widget.value = f"Converting file number: {index_num}"
    pil_image = load_image(image_id)
    input_tensor = preprocess(pil_image).unsqueeze(0)
    if torch.cuda.is_available():
        input_tensor = input_tensor.to('cuda')

    embedding = embedding_model(input_tensor)
    return embedding[0].squeeze().cpu()

def get_df_embeddings(pictures_df, embedding_model, preprocess):
    display_out = ipywidgets.HTML()
    display(display_out)

    if torch.cuda.is_available():
        embedding_model.to('cuda')

    split_dfs = np.array_split(pictures_df, DF_SPLITS)
    with torch.no_grad():
        for df_split in tqdm(split_dfs):
            df_split["embeddings"] = df_split.apply(lambda row: image_to_embedding(preprocess, embedding_model, row["id"], display_out, row.name), axis=1)
    embedding_df = pd.concat(split_dfs)
    return embedding_df



In [ ]:
for config in RUN_CONFIGS:
    embedding_model = config["model"]()
    to_embeddings_df = pictures_df.copy()
    embedding_df = get_df_embeddings(to_embeddings_df, embedding_model, preprocess)
    embedding_df.to_pickle(EMBEDDINGS_SAVE_DIR + config["save_file_name"])

In [ ]:
test_model = load_resnet18_model()
test_df = pictures_df.sample(200)
output_df = get_df_embeddings(test_df, test_model, preprocess)
output_df.head()

In [79]:
#torch.dist(output_df.iloc[0]["embeddings"], output_df.iloc[90]["embeddings"])
start_point = output_df["embeddings"].iloc[2]
output_df["distances_to_first"] = output_df["embeddings"].apply(lambda x: torch.dist(start_point, x).item())
display(output_df["distances_to_first"].describe())
output_df

count    200.000000
mean       2.192919
std        0.166880
min        0.000000
25%        2.164817
50%        2.197933
75%        2.233671
max        2.422177
Name: distances_to_first, dtype: float64

,id,owner,embeddings,distances_to_first
6022,picture.55d40dd7b3a541abbb94d7e2e24338ef,outfit.570f138cb9aa4b81a76b56190c107bac,"[tensor(0.5149), tensor(0.4810), tensor(0.7695...",2.311112
10937,picture.9c71e18b685842938e86d3d156ca29d7,outfit.a743bd93e1a86501,"[tensor(0.5660), tensor(0.4256), tensor(0.7989...",2.204357
3901,picture.3768419180bb4674879e8fbce34826c2,outfit.d7f1bc7197b6469db1b9c4faf4071cb7,"[tensor(0.5320), tensor(0.4571), tensor(0.8046...",0.000000
17576,picture.f8be256209814145b728193d964f2a0b,outfit.d720fa6c17bf4c688054cbda17336d6c,"[tensor(0.5729), tensor(0.4188), tensor(0.7448...",2.197646
12076,picture.aab941d75a2c4e9daddabdcd1b19409b,outfit.c16aba693fc6409c81937fa356d4f3ee,"[tensor(0.5748), tensor(0.4459), tensor(0.7536...",2.170306
...,...,...,...,...
5917,picture.548e2396d6324ec98253addbf7de9773,outfit.8dd4f2aae17b4a83817027414ba7d14f,"[tensor(0.5510), tensor(0.4328), tensor(0.7743...",2.171227
428,picture.05d91fac40b745ebb2af0fe67f8a8e75,outfit.c5805456353a4ee9bd110ccf3442e753,"[tensor(0.4767), tensor(0.4310), tensor(0.8078...",2.248994
4555,picture.40f584a646784183b67d9e2033f0f16d,outfit.ce1df3bc87204493b2e4bd0b0973a576,"[tensor(0.5514), tensor(0.4343), tensor(0.7182...",2.164786
1194,picture.10b38e93cc3c442ea23dc11200771237,outfit.9481d2d7251be70f,"[tensor(0.5763), tensor(0.4468), tensor(0.7194...",2.265918
